In [ ]:
import numpy as np
import cv2
from scipy.ndimage import maximum_filter, minimum_filter


def fabemd(image, K):
    def extract_imf(image, sizeK = 3):
        # Find local maxima and minima
        max_env = maximum_filter(image, size=sizeK)
        min_env = minimum_filter(image, size=sizeK)

        # Calculate the mean envelope
        mean_env = (max_env + min_env) / 2

        # Extract the IMF
        imf = image - mean_env
        return imf, mean_env

    BIMFs = []
    residual = image.astype(np.float64)

    for ik in range(K):
        imf, mean_env = extract_imf(residual, sizeK=ik)
        BIMFs.append(imf)
        residual -= imf

    return BIMFs, residual


def decompose_image(image_path, K=10):
    # Load the image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        raise ValueError("Image not found or unable to load.")

    # Perform FABEMD
    BIMFs, residual = fabemd(image, K)

    # Output the results
    for i, bimf in enumerate(BIMFs):
        cv2.imwrite(f"BIMF_{i+1}.png", bimf.astype(np.uint8))
    cv2.imwrite("residual.png", residual.astype(np.uint8))

    return BIMFs, residual


# Example usage
image_path = "images/1-IMA-01B_Thorax_AP.tiff"
BIMFs, residual = decompose_image(image_path)